<a href="https://colab.research.google.com/github/idcesares/Data-Science-Mini-Projects/blob/main/FRANKIE_OFICINA_3_VERS%C3%83O_COM_C%C3%93DIGOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Oficina 3

Olá pessoal! Bem vindos ao nosso Google Colab!
Este ambiente é o Python ! Muito cuidado com a Indentação !

Nesta atividade você irá:
* 01. Instalar a Wisard
* 02. Importar bibliotecas
* 03. Acessar pasta no Google Drive pelo Google Colab
* 04. Definir o número de endereços de cada neurônio
* 05. Organizar os elementos de treino da Wisard - Transformar imagens em vetores
* 06. Treinar a Wisard
* 07. Acessar a Imagem fotografada
* 08. Tratar a imagem fotografada - Transformar imagem em vetor
* 09. Checar a imagem de teste
* 10. Escrever os resultados da classificação

PRIMEIRA CÉLULA: **INSTALANDO A WISARD**

In [ ]:
pip install wisardpkg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


SEGUNDA CÉLULA: **IMPORTANDO BIBLIOTECAS**

In [ ]:
#!/usr/bin/env python
from __future__ import print_function
import wisardpkg as wp
import cv2
import PIL
import numpy
import time
import sys

TERCEIRA CÉLULA: **ACESSAR PASTA COM AS IMAGENS DE TREINO E TESTE NO GOOGLE DRIVE PELO GOOGLE COLAB**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

**>>> ATIVIDADES DE TREINO <<<**

QUARTA CÉLULA: **DEFINIR O TAMANHO DOS ENDEREÇOS i DE CADA NEURÔNIO**

In [ ]:
#Wisard
addressSize = 8
ignoreZero  = False
verbose = True
wsd = wp.Wisard(addressSize, ignoreZero=ignoreZero, verbose=verbose,returnConfidence=True)

QUINTA CÉLULA: **DEFININDO O TREINO DA WISARD** Observe que a ordem é X - Vetores e y - Rótulos

---

Obs.: Com a mesma imagem capturada, se dermos o play diferentes vezes na célula de treino, obteremos valores de confiança diferentes no final. Isso acontece por que a Wisard escolhe aleatoriamente os pixels toda vez que é treinada e isso altera o resultado.

In [ ]:
def switch_class_name(class_number):
    switcher = {1: "Quadrado", 2: "Triângulo", 3: "Circulo", 4: "Estrela", 5: "Seta",}  # Os rótulos  
    return switcher.get(class_number)
  
def train_wisard (wisard):
    X=[]
    y=[]
    #print ("Treinando Wisard...")
    for i in range(1,6):   # Alterar de acordo com o número de classes
        for j in range(1,11):  # Alterar de acordo com o número de imagens de treino
            print ("carregando classe " + str(i) +" exemplar " + str(j))
            file = '/content/gdrive/My Drive/Colab Notebooks/projetofrankieoficina3/quatro_classes_scan/classe_'+str(i) +'/picture_' + str(j)+'.bmp'
            img = cv2.imread(file,0)
            img = cv2.resize(img,(100, 100)) # Número de linhas e colunas da retina. Neste caso, são 10000 pixels. Pode ser alterado.
            input_data = [(1 if e==255 else 0) for e in img.flatten()] # Aqui a imagem vira vetor !
            X.append(input_data)
            y.append(switch_class_name (i))
            wisard.train(X,y)

SEXTA CÉLULA: **TREINANDO A WISARD** 

In [ ]:
train_wisard(wsd)

**>>> ATIVIDADES DE TESTE <<<**

SÉTIMA CÉLULA: **ACESSANDO A IMAGEM FOTOGRAFADA**

In [ ]:
def get_pic_and_rec():
    Z=[]
    print("Obtendo imagem...")
    
                                  #TROCAR O NOME PARA O NOME DA NOVA IMAGEM
    
    testFile = PIL.Image.open ('/content/gdrive/My Drive/Colab Notebooks/projetofrankieoficina3/teste3.jpg')
    
                                  #CONTINUANDO
    
    image_to_rec = process_image (testFile) #captura a imagem

    #transforma a imagem da openCV em um vetor de entrada para WiSARD
    input_data = [(1 if e==255 else 0) for e in image_to_rec.flatten()]

    #time.sleep(2)  #espera 2 segundos pra mostrar a resposta (retirar - usado nos testes para facilitar visualização da saída)

    Z.append(input_data)

    dn=0  #bacalhau pra ver se gerou uma imagem toda preta na hora da transformação
    for e in image_to_rec.flatten():
        if e != 0:
            dn=1
            break
    if dn == 0:
        return ["nao sei","indefinida"]
    else: #senão gerou uma imagem toda preta, diz o que reconheceu
        image_id_conf = recognize(Z)
        #Verifica confiança da imagem
        if image_id_conf[1]< 0.0:
            return ["nao sei", image_id_conf[1]]
        else:
            return recognize(Z)

OITAVA CÉLULA: **TRATANDO A IMAGEM FOTOGRAFADA**

In [ ]:
def process_image(initial_image):
    
    #Transforma a imagem em preto e branco
    image_gray = cv2.cvtColor(numpy.array(initial_image), cv2.COLOR_BGR2GRAY)
    _,image_thresh = cv2.threshold(image_gray,50,255,cv2.THRESH_BINARY)
    image_risized = cv2.resize(image_thresh,(100,100))
    
    #escreve a imagem no disco para poder inverter (bacalhau)
    cv2.imwrite("/content/gdrive/My Drive/Colab Notebooks/projetofrankieoficina3/image.bmp", image_risized) 
    
    #inverte a imagem
    inverted_image= cv2.imread("/content/gdrive/My Drive/Colab Notebooks/projetofrankieoficina3/image.bmp")
    inverted_image[inverted_image == 255] = 1
    inverted_image[inverted_image == 0] = 255
    inverted_image[inverted_image == 1] = 0
    
    #prepara a imagem para ser usada no findContours (bacalhau)
    inverted_image_gray = cv2.cvtColor(inverted_image,cv2.COLOR_BGR2GRAY)
    ret,inverted_image_tresh = cv2.threshold(inverted_image_gray,127,255,0)
    #so controle para ver o resultado da trasnformaçao
    cv2.imwrite("/content/gdrive/My Drive/Colab Notebooks/projetofrankieoficina3/inverted_image.bmp", inverted_image_tresh) 
    
    #encontra as coordenadas da figura geometrica dentro da imagem capturada
    #contours, hierarchy = cv2.findContours(inverted_image_tresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    im2,contours, hierarchy = cv2.findContours(inverted_image_tresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]
    x,y,w,h = cv2.boundingRect(cnt)
    
    #recorda e cria uma nova imagem somente com a figura geometrica
    ret,inverted_image = cv2.threshold(inverted_image,0,255,cv2.THRESH_BINARY_INV)  
    detected_image = inverted_image[y:y+h,x:x+w]
    
    detected_image = cv2.cvtColor(detected_image,cv2.COLOR_BGR2GRAY)
    
    _,detected_image = cv2.threshold(detected_image,50,255,cv2.THRESH_BINARY)       
    
    #redimensiona a imagem
    detected_image_risized = cv2.resize(detected_image,(100, 100))
        
    #so controle para ver o resultado da trasnformaçao
    cv2.imwrite("/content/gdrive/My Drive/Colab Notebooks/projetofrankieoficina3/detected_image_risized.bmp", detected_image_risized)               
        
    return detected_image_risized

NONA CÉLULA: **CHECANDO A IMAGEM DE TESTE**

In [ ]:
def recognize (vector):
    out = wsd.classify(vector)
    
    return [out[0]["class"],out[0]["confidence"]]

DÉCIMA CÉLULA: **ESCREVENDO O RESULTADO DA CLASSIFICAÇÃO**

In [ ]:
#acessa a foto no Google Drive para reconhecer com a WiSARD
entrada = input("Tecle t para testar ou s para sair e depois tecle enter: ")
while entrada != "s":
    image_id_conf = get_pic_and_rec()
    print ("Forma determinada: " + str( image_id_conf[0]) + " - Confiança: %.6s" % (image_id_conf[1] * 100) + "%")
    entrada = input("Tecle t para testar ou s para sair e depois tecle enter: ")

Tecle t para testar ou s para sair e depois tecle enter: t
Obtendo imagem...
Forma determinada: Triângulo - Confiança: 95.646%
